In [ ]:
!pip install torch torchtext spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [ ]:
# ==============================================================================
# PROJECT: Neural Machine Translation (German -> English) using Transformers
# ==============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import math
import random
import time
import spacy
import tarfile
import os
import urllib.request
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from typing import Iterable, List

# ==========================================
# 0. INSTALLATION & SETUP
# ==========================================
# Run system commands to install dependencies if missing
try:
    import spacy
    spacy.load('en_core_web_sm')
except OSError:
    print("Installing Spacy models...")
    os.system("python -m spacy download en_core_web_sm")
    os.system("python -m spacy download de_core_news_sm")

# Set Seed for Reproducibility
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ==========================================
# 1. CUSTOM DATA LOADING (BYPASSING TORCHTEXT)
# ==========================================
print("Phase 1: Downloading & Loading Data Manually...")

def download_and_extract(url, filename):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
    if not os.path.exists(filename.replace(".tar.gz", "")):
        print(f"Extracting {filename}...")
        with tarfile.open(filename, "r:gz") as tar:
            tar.extractall()

# URLs for the Multi30k dataset (Backup Mirror)
urls = {
    "train": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz",
    "val": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz",
    "test": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt_task1_test2016.tar.gz"
}

download_and_extract(urls["train"], "training.tar.gz")
download_and_extract(urls["val"], "validation.tar.gz")
download_and_extract(urls["test"], "mmt_task1_test2016.tar.gz")

def read_file(path):
    with open(path, encoding='utf-8') as f:
        return [line.strip() for line in f]

# Load raw sentences into lists (Source: German, Target: English)
train_src = read_file("train.de")
train_tgt = read_file("train.en")
val_src = read_file("val.de")
val_tgt = read_file("val.en")
test_src = read_file("test2016.de")
test_tgt = read_file("test2016.en")

# Combine into tuples (German, English)
train_list = list(zip(train_src, train_tgt))
valid_list = list(zip(val_src, val_tgt))
test_list = list(zip(test_src, test_tgt))

print(f"Data Loaded: {len(train_list)} Training, {len(valid_list)} Validation, {len(test_list)} Testing sentences.")

# ==========================================
# 2. PREPROCESSING & VOCABULARY
# ==========================================
print("\nPhase 2: Building Vocabulary...")
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Load Tokenizers
token_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

# Special Tokens
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

def yield_tokens(data_iter, language):
    for data_sample in data_iter:
        raw_text = data_sample[0] if language == SRC_LANGUAGE else data_sample[1]
        yield token_transform[language](raw_text)

vocab_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln] = build_vocab_from_iterator(
        yield_tokens(train_list, ln), min_freq=2, specials=special_symbols, special_first=True
    )
    vocab_transform[ln].set_default_index(UNK_IDX)

print(f"Vocab Size -> German: {len(vocab_transform['de'])}, English: {len(vocab_transform['en'])}")

# ==========================================
# 3. DATA PIPELINE
# ==========================================
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], vocab_transform[ln], tensor_transform)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

# ==========================================
# 4. TRANSFORMER MODEL ARCHITECTURE
# ==========================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward, dropout=dropout)
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.d_model = d_model

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_embedding(src) * math.sqrt(self.d_model))
        tgt_emb = self.positional_encoding(self.tgt_embedding(tgt) * math.sqrt(self.d_model))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.fc_out(outs)

# ==========================================
# 5. TRAINING SETUP & EXECUTION
# ==========================================
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
DROPOUT = 0.1
NUM_EPOCHS = 5

model = TransformerModel(SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, EMB_SIZE, NHEAD,
                         NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, FFN_HID_DIM, DROPOUT).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_dataloader = torch.utils.data.DataLoader(train_list, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src, tgt = src.to(device), tgt.to(device)
        tgt_input = tgt[:-1, :]

        src_mask = torch.zeros((src.shape[0], src.shape[0])).type(torch.bool).to(device)
        tgt_mask = model.generate_square_subsequent_mask(tgt_input.shape[0]).to(device)

        src_padding_mask = (src == PAD_IDX).transpose(0, 1)
        tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1)
        memory_key_padding_mask = src_padding_mask

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)

        optimizer.zero_grad()
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses / len(train_dataloader)

print(f"\nPhase 3: Starting Training on {device}...")
for epoch in range(1, NUM_EPOCHS + 1):
    start_time = time.time()
    train_loss = train_epoch(model, optimizer)
    print(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, Time: {(time.time() - start_time):.2f}s")

# ==========================================
# 6. FINAL DEMO (INFERENCE)
# ==========================================
def translate(model, src_sentence):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1).to(device)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)

    memory = model.transformer.encoder(model.positional_encoding(model.src_embedding(src) * math.sqrt(EMB_SIZE)), src_mask)
    ys = torch.ones(1, 1).fill_(BOS_IDX).type(torch.long).to(device)

    for i in range(20):
        tgt_mask = (model.generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(device)
        out = model.transformer.decoder(model.positional_encoding(model.tgt_embedding(ys) * math.sqrt(EMB_SIZE)), memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.fc_out(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break

    tokens = [vocab_transform[TGT_LANGUAGE].lookup_token(i) for i in ys.flatten() if i not in [BOS_IDX, EOS_IDX]]
    return " ".join(tokens)

print("\n=== FINAL DEMO ===")
examples = [
    "Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen",
    "Ein Mann schläft in einem grünen Raum",
    "Zwei Frauen gehen spazieren"
]

for ex in examples:
    print(f"DE: {ex}")
    print(f"AI: {translate(model, ex)}")
    print("-" * 20)

/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

Phase 1: Downloading & Loading Data Manually...
Extracting training.tar.gz...
Extracting validation.tar.gz...
Extracting mmt_task1_test2016.tar.gz...
Data Loaded: 29001 Training, 1015 Validation, 1000 Testing sentences.

Phase 2: Building Vocabulary...


/tmp/ipython-input-286101890.py:52: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


Vocab Size -> German: 8014, English: 6191


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")



Phase 3: Starting Training on cpu...


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
